## Create Data Maps from NYSDEC Permit Applications

In [ ]:
from GeoCodes import *
% matplotlib notebook

In [ ]:
#---Read in data to begin analysis

#---Permit data
f = r'permit_intersect.txt'
data = pd.read_csv(f, sep = ',')

#---Flow data
f_flow = r'B_NormalFlows_int.txt'
flow_data = pd.read_csv(f_flow, sep = '\t')

#---Structure data
huc12 = r'hucs_bridges.txt'
huc12_data = pd.read_csv(huc12, sep = '\t')

In [ ]:
#-- Count Permits by type from list:
'''
DA = Dams and Impoundments; 
DO = Docks and Moorings; 
EF = Excavation and Fill in Navigable Waters; 
SD = Stream Disturbance of Protected Stream; 
WQ = Water Quality Certification
'''

p_types = data['DEC_Jurisd'].unique()
p_total = 0 #--Added as a check to make sure the numbers make sense 

for p in p_types:
    permits = (data['DEC_Jurisd']== p).sum()
    #print(p, permits)
    p_total += permits

#print('\nTotal Permits = ', p_total)

In [ ]:
#--Select subset of permits: Stream Disturbance only

df_SD = data[data['DEC_Jurisd']== 'SD']

#--Get a count of permits by 'Project' (Batch_Numb field) 
b_types = df_SD['Batch_Numb'].unique() 

print('Stream Disturbance of Protected Stream: ', len(df_SD), 'records', len(b_types),'Projects')

In [ ]:
#-- Get a count of permits by 'HUC 12'

df_HUC = df_SD.groupby('HUC_12').count()

b_types = df_SD['Batch_Numb'].unique()

#-- Rank the HUC 12's by number of permits 

df_HUC_sorted = df_HUC.sort_values(by = ['Batch_Numb'], ascending=False)

print('Stream Disturbance of Protected Stream: ', len(df_HUC), 'HUCs', len(b_types),'Projects')

In [ ]:
# Select fields from the Permit dataset to include in a map

usecols = ['x','y','Short_Proj','owner'] #--all columns after x,y used in map's Pop-up

data_popup = usecols[2:len(usecols)] 

data[usecols].head(2)

In [ ]:
#-- Merge datasets

flow_data[['BIN']] = flow_data[['BIN']].astype(str)
huc12_data[['BIN']] = huc12_data[['BIN']].astype(str)

df_total = pd.merge(flow_data,huc12_data, on = 'BIN', how = 'outer')
df_total.head()
flow_data.head()
huc12_data.head()
#df_total.head(2)

In [ ]:
# Select fields from the Permit dataset to include in a map

usecols2 = ['x','y','BIN','Q100', 'HUC_12'] #--all columns after x,y used in map's Pop-up

data_popup2 = usecols2[2:len(usecols2)] 

df_total[usecols2].head(2)

In [ ]:
field = 'HUC_12'

item = 20200060905


df = data.loc[data[field] == item]
df2 = df_total.loc[df_total[field] == item]
xbar = df['x'].mean()
ybar = df['y'].mean()
geoj =df_to_geojson(df, data_popup, lat= 'y', lon='x')
geoj2 =df_to_geojson(df2, data_popup2, lat= 'y', lon='x')

newmap = '{}_{}.html'.format(field,str(item))
map_template = 'map_template.html'

mapit(newmap,geoj,geoj2,xbar,ybar,map_template)